In [ ]:
-- Cell 1: Database and Schema Setup (Snowflake)
-- ==============================================
-- This notebook creates an automobile database schema with three tables:
-- - Vehicles: Vehicle information linked to VINs
-- - Owners: Owner information distributed by US state population
-- - Repair_Records: Repair history (Toyota Camry has 20% more repairs)

-- Create Database (run with appropriate privileges)

SET DATABASE_NAME = 'FORD_AUTOMOTIVE_DB';
CREATE DATABASE IF NOT EXISTS IDENTIFIER($DATABASE_NAME);

-- Use the database
USE DATABASE IDENTIFIER($DATABASE_NAME);

-- Create Schema
CREATE SCHEMA IF NOT EXISTS AUTOMOBILE_SCHEMA;

-- Use the schema
USE SCHEMA AUTOMOBILE_SCHEMA;

-- Verify setup
SELECT CURRENT_DATABASE(), CURRENT_SCHEMA();


In [ ]:
-- Cell 2: Table Creation
-- =======================

-- Drop tables if they exist (for clean re-runs)
DROP TABLE IF EXISTS Repair_Records;
DROP TABLE IF EXISTS Owners;
DROP TABLE IF EXISTS Vehicles;

-- Create Vehicles table
CREATE TABLE Vehicles (
    vin VARCHAR(17) PRIMARY KEY,
    make VARCHAR(50) NOT NULL,
    model VARCHAR(50) NOT NULL,
    year INTEGER NOT NULL,
    color VARCHAR(30),
    engine_type VARCHAR(30),
    transmission VARCHAR(30),
    created_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);

-- Create Owners table
CREATE TABLE Owners (
    owner_id INTEGER AUTOINCREMENT PRIMARY KEY,
    vin VARCHAR(17) NOT NULL,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100),
    phone VARCHAR(20),
    address VARCHAR(200),
    city VARCHAR(100),
    state VARCHAR(2) NOT NULL,
    zip_code VARCHAR(10),
    purchase_date DATE,
    FOREIGN KEY (vin) REFERENCES Vehicles(vin)
);

-- Create Repair_Records table
CREATE TABLE Repair_Records (
    repair_id INTEGER AUTOINCREMENT PRIMARY KEY,
    vin VARCHAR(17) NOT NULL,
    repair_date DATE NOT NULL,
    repair_type VARCHAR(50) NOT NULL,
    description VARCHAR(500),
    mileage INTEGER,
    cost DECIMAL(10, 2),
    dealer_name VARCHAR(100),
    warranty_claim BOOLEAN DEFAULT FALSE,
    FOREIGN KEY (vin) REFERENCES Vehicles(vin)
);

-- Verify tables created
SHOW TABLES;


In [ ]:
# Cell 3: Vehicles Synthetic Data
# =================================
# Generates synthetic vehicle data and inserts into Snowflake

import random
import pandas as pd
from datetime import datetime, timedelta

# Load VINs from file
with open("vins.txt", "r") as f:
    vins = [line.strip() for line in f.readlines()]

print(f"Loaded {len(vins)} VINs from vins.txt")

# Vehicle makes and models (non-Ford brands)
# Toyota Camry is designated as the "problematic" model with 20% more repairs
MAKES_MODELS = {
    "Ford": ["F-150", "Mustang", "Explorer", "Escape", "Bronco", "Edge", "Ranger"],
    "Toyota": ["Camry", "Corolla", "RAV4", "Highlander", "Tacoma"],  # Camry = problem model
    "Honda": ["Civic", "Accord", "CR-V", "Pilot", "HR-V"],
    "Chevrolet": ["Silverado", "Equinox", "Malibu", "Tahoe", "Colorado"],
    "Nissan": ["Altima", "Rogue", "Sentra", "Pathfinder", "Frontier"],
    "Hyundai": ["Elantra", "Sonata", "Tucson", "Santa Fe", "Palisade"],
    "BMW": ["3 Series", "5 Series", "X3", "X5"],
    "Mercedes-Benz": ["C-Class", "E-Class", "GLC", "GLE"]
}

COLORS = ["White", "Black", "Silver", "Gray", "Red", "Blue", "Green", "Brown", "Beige"]
ENGINE_TYPES = ["Gasoline", "Diesel", "Hybrid", "Electric", "Plug-in Hybrid"]
TRANSMISSIONS = ["Automatic", "Manual", "CVT", "Dual-Clutch"]

# Generate vehicle data
vehicles_data = []
random.seed(42)

for vin in vins:
    make = random.choice(list(MAKES_MODELS.keys()))
    model = random.choice(MAKES_MODELS[make])
    year = random.randint(2015, 2024)
    color = random.choice(COLORS)
    engine_type = random.choice(ENGINE_TYPES)
    transmission = random.choice(TRANSMISSIONS)
    
    vehicles_data.append({
        "vin": vin,
        "make": make,
        "model": model,
        "year": year,
        "color": color,
        "engine_type": engine_type,
        "transmission": transmission
    })

# Create DataFrame
vehicles_df = pd.DataFrame(vehicles_data)

# Display sample
print(f"\n✓ Generated {len(vehicles_df)} vehicle records")
print("\nSample data:")
display(vehicles_df.head(10))

# Show make/model distribution
print("\nMake distribution:")
print(vehicles_df['make'].value_counts())

# Check Toyota Camry count
camry_count = len(vehicles_df[(vehicles_df['make'] == 'Toyota') & (vehicles_df['model'] == 'Camry')])
print(f"\n⚠️  Toyota Camry count: {camry_count} (designated for 20% more repairs)")


In [ ]:
# Cell 3b: Insert Vehicles into Snowflake using Snowpark
# ========================================================
from snowflake.snowpark.context import get_active_session

# Get active Snowpark session (available in Snowflake Notebooks)
session = get_active_session()

# Get current database from session (set by SQL cell)
# Strip quotes that Snowflake adds to identifiers
DATABASE_NAME = session.get_current_database().strip('"')
SCHEMA_NAME = session.get_current_schema().strip('"')
print(f"Using database: {DATABASE_NAME}, schema: {SCHEMA_NAME}")

# Rename columns to match table schema (uppercase for Snowflake)
vehicles_sf = vehicles_df.rename(columns={
    'vin': 'VIN',
    'make': 'MAKE',
    'model': 'MODEL',
    'year': 'YEAR',
    'color': 'COLOR',
    'engine_type': 'ENGINE_TYPE',
    'transmission': 'TRANSMISSION'
})

# Write DataFrame to Snowflake table using Snowpark
session.write_pandas(
    vehicles_sf, 
    table_name="VEHICLES",
    database=DATABASE_NAME,
    schema=SCHEMA_NAME,
    auto_create_table=False,
    overwrite=True
)

print(f"✓ Inserted {len(vehicles_df)} vehicles into VEHICLES table using Snowpark")


In [ ]:
# Cell 4: Owners Synthetic Data (Distributed by State Population)
# =================================================================

# US State populations (2023 estimates, in millions) for weighted distribution
STATE_POPULATIONS = {
    "CA": 39.0, "TX": 30.0, "FL": 22.2, "NY": 19.7, "PA": 12.9,
    "IL": 12.6, "OH": 11.8, "GA": 10.9, "NC": 10.7, "MI": 10.0,
    "NJ": 9.3, "VA": 8.6, "WA": 7.8, "AZ": 7.4, "MA": 7.0,
    "TN": 7.0, "IN": 6.8, "MO": 6.2, "MD": 6.2, "WI": 5.9,
    "CO": 5.8, "MN": 5.7, "SC": 5.3, "AL": 5.1, "LA": 4.6,
    "KY": 4.5, "OR": 4.2, "OK": 4.0, "CT": 3.6, "UT": 3.4,
    "IA": 3.2, "NV": 3.2, "AR": 3.0, "MS": 2.9, "KS": 2.9,
    "NM": 2.1, "NE": 2.0, "ID": 1.9, "WV": 1.8, "HI": 1.4,
    "NH": 1.4, "ME": 1.4, "MT": 1.1, "RI": 1.1, "DE": 1.0,
    "SD": 0.9, "ND": 0.8, "AK": 0.7, "VT": 0.6, "WY": 0.6
}

# Create weighted list of states
states_weighted = []
for state, pop in STATE_POPULATIONS.items():
    states_weighted.extend([state] * int(pop * 10))

# Sample first names and last names
FIRST_NAMES = [
    "James", "Mary", "John", "Patricia", "Robert", "Jennifer", "Michael", "Linda",
    "William", "Elizabeth", "David", "Barbara", "Richard", "Susan", "Joseph", "Jessica",
    "Thomas", "Sarah", "Charles", "Karen", "Christopher", "Nancy", "Daniel", "Lisa",
    "Matthew", "Betty", "Anthony", "Margaret", "Mark", "Sandra", "Donald", "Ashley",
    "Steven", "Kimberly", "Paul", "Emily", "Andrew", "Donna", "Joshua", "Michelle",
    "Carlos", "Maria", "Jose", "Rosa", "Miguel", "Carmen", "Luis", "Sofia",
    "Wei", "Li", "Raj", "Priya", "Ahmed", "Fatima", "Yuki", "Sakura"
]

LAST_NAMES = [
    "Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis",
    "Rodriguez", "Martinez", "Hernandez", "Lopez", "Gonzalez", "Wilson", "Anderson",
    "Thomas", "Taylor", "Moore", "Jackson", "Martin", "Lee", "Perez", "Thompson",
    "White", "Harris", "Sanchez", "Clark", "Ramirez", "Lewis", "Robinson", "Walker",
    "Young", "Allen", "King", "Wright", "Scott", "Torres", "Nguyen", "Hill",
    "Kim", "Chen", "Wang", "Patel", "Singh", "Kumar", "Shah", "Ali"
]

# City data by state
CITIES_BY_STATE = {
    "CA": ["Los Angeles", "San Francisco", "San Diego", "San Jose", "Sacramento", "Fresno"],
    "TX": ["Houston", "Dallas", "Austin", "San Antonio", "Fort Worth", "El Paso"],
    "FL": ["Miami", "Orlando", "Tampa", "Jacksonville", "Fort Lauderdale", "Naples"],
    "NY": ["New York City", "Buffalo", "Rochester", "Syracuse", "Albany", "Yonkers"],
    "PA": ["Philadelphia", "Pittsburgh", "Allentown", "Erie", "Reading", "Scranton"],
    "IL": ["Chicago", "Aurora", "Naperville", "Joliet", "Rockford", "Springfield"],
    "OH": ["Columbus", "Cleveland", "Cincinnati", "Toledo", "Akron", "Dayton"],
    "GA": ["Atlanta", "Augusta", "Columbus", "Savannah", "Athens", "Macon"],
}

# Default cities for states not in detailed list
DEFAULT_CITIES = ["Springfield", "Franklin", "Clinton", "Madison", "Georgetown"]

STREET_NAMES = ["Main", "Oak", "Maple", "Cedar", "Pine", "Elm", "Washington", "Lincoln", "Park", "Lake"]
STREET_TYPES = ["St", "Ave", "Blvd", "Dr", "Ln", "Way", "Rd"]

# Generate owner data
owners_data = []
random.seed(43)

for vin in vins:
    state = random.choice(states_weighted)
    cities = CITIES_BY_STATE.get(state, DEFAULT_CITIES)
    city = random.choice(cities)
    
    first_name = random.choice(FIRST_NAMES)
    last_name = random.choice(LAST_NAMES)
    email = f"{first_name.lower()}.{last_name.lower()}{random.randint(1, 999)}@email.com"
    phone = f"{random.randint(200, 999)}-{random.randint(200, 999)}-{random.randint(1000, 9999)}"
    address = f"{random.randint(100, 9999)} {random.choice(STREET_NAMES)} {random.choice(STREET_TYPES)}"
    zip_code = f"{random.randint(10000, 99999)}"
    
    # Purchase date within last 5 years
    days_ago = random.randint(0, 5 * 365)
    purchase_date = (datetime.now() - timedelta(days=days_ago)).strftime("%Y-%m-%d")
    
    owners_data.append({
        "vin": vin,
        "first_name": first_name,
        "last_name": last_name,
        "email": email,
        "phone": phone,
        "address": address,
        "city": city,
        "state": state,
        "zip_code": zip_code,
        "purchase_date": purchase_date
    })

# Create DataFrame
owners_df = pd.DataFrame(owners_data)

# Display sample
print(f"✓ Generated {len(owners_df)} owner records")
print("\nSample data:")
display(owners_df.head(10))

# Show state distribution
print("\nTop 10 States by owner count:")
print(owners_df['state'].value_counts().head(10))


In [ ]:
# Cell 4b: Insert Owners into Snowflake using Snowpark
# ======================================================

# Rename columns to match table schema (uppercase for Snowflake)
owners_sf = owners_df.rename(columns={
    'vin': 'VIN',
    'first_name': 'FIRST_NAME',
    'last_name': 'LAST_NAME',
    'email': 'EMAIL',
    'phone': 'PHONE',
    'address': 'ADDRESS',
    'city': 'CITY',
    'state': 'STATE',
    'zip_code': 'ZIP_CODE',
    'purchase_date': 'PURCHASE_DATE'
})

# Write DataFrame to Snowflake table using Snowpark
session.write_pandas(
    owners_sf, 
    table_name="OWNERS",
    database=DATABASE_NAME,
    schema=SCHEMA_NAME,
    auto_create_table=False,
    overwrite=True
)

print(f"✓ Inserted {len(owners_df)} owners into OWNERS table using Snowpark")


In [ ]:
# Cell 5: Repair Records Synthetic Data
# =======================================
# Toyota Camry has 20% more repairs than other vehicles

# Repair types and descriptions
REPAIR_TYPES = {
    "Engine": [
        "Oil change and filter replacement",
        "Spark plug replacement",
        "Timing belt replacement",
        "Engine tune-up",
        "Coolant flush and refill",
        "Engine mount replacement"
    ],
    "Transmission": [
        "Transmission fluid change",
        "Clutch replacement",
        "Transmission rebuild",
        "Shift cable adjustment"
    ],
    "Brakes": [
        "Brake pad replacement",
        "Brake rotor resurfacing",
        "Brake fluid flush",
        "Brake caliper replacement",
        "ABS sensor replacement"
    ],
    "Electrical": [
        "Battery replacement",
        "Alternator replacement",
        "Starter motor replacement",
        "Fuse box repair",
        "Wiring harness repair"
    ],
    "Suspension": [
        "Shock absorber replacement",
        "Strut replacement",
        "Control arm replacement",
        "Wheel alignment",
        "Ball joint replacement"
    ],
    "HVAC": [
        "AC recharge",
        "AC compressor replacement",
        "Heater core replacement",
        "Blower motor replacement"
    ],
    "Recall": [
        "Safety recall - airbag inspection",
        "Safety recall - fuel system",
        "Safety recall - software update",
        "Safety recall - brake system"
    ]
}

DEALERS = [
    "AutoNation Ford", "Galpin Motors", "Hendrick Automotive", "Larry H. Miller",
    "Sewell Automotive", "Park Place Dealerships", "Fields Auto Group",
    "Toyota of San Diego", "Honda World", "Jim Ellis Automotive",
    "CarMax Service Center", "Jiffy Lube", "Midas Auto Service", "Pep Boys"
]

# Cost ranges by repair type
COST_RANGES = {
    "Engine": (150, 2500),
    "Transmission": (200, 4000),
    "Brakes": (100, 800),
    "Electrical": (50, 500),
    "Suspension": (150, 1200),
    "HVAC": (100, 1500),
    "Recall": (0, 0)  # Recalls are free
}

# Generate repair records
repairs_data = []
random.seed(44)

# Base number of repairs per vehicle (average 2-5 repairs)
BASE_REPAIRS_MIN = 2
BASE_REPAIRS_MAX = 5

# 20% more repairs for Toyota Camry
PROBLEM_MODEL_MULTIPLIER = 1.2

for _, vehicle in vehicles_df.iterrows():
    vin = vehicle['vin']
    make = vehicle['make']
    model = vehicle['model']
    year = vehicle['year']
    
    # Calculate number of repairs for this vehicle
    base_repairs = random.randint(BASE_REPAIRS_MIN, BASE_REPAIRS_MAX)
    
    # Apply 20% more repairs for Toyota Camry (non-Ford problematic model)
    if make == "Toyota" and model == "Camry":
        num_repairs = int(base_repairs * PROBLEM_MODEL_MULTIPLIER) + random.randint(0, 1)
    else:
        num_repairs = base_repairs
    
    # Generate repairs over the vehicle's lifetime
    vehicle_age_days = (2024 - year + 1) * 365
    
    for _ in range(num_repairs):
        repair_type = random.choice(list(REPAIR_TYPES.keys()))
        description = random.choice(REPAIR_TYPES[repair_type])
        
        # Repair date within vehicle's lifetime
        days_ago = random.randint(30, max(31, vehicle_age_days))
        repair_date = (datetime.now() - timedelta(days=days_ago)).strftime("%Y-%m-%d")
        
        # Mileage correlates with age
        base_mileage = (2024 - year) * random.randint(10000, 15000)
        mileage = max(1000, base_mileage + random.randint(-5000, 10000))
        
        # Cost varies by repair type
        min_cost, max_cost = COST_RANGES[repair_type]
        cost = round(random.uniform(min_cost, max_cost), 2) if max_cost > 0 else 0.00
        
        dealer = random.choice(DEALERS)
        warranty_claim = repair_type == "Recall" or (year >= 2021 and random.random() < 0.3)
        
        repairs_data.append({
            "vin": vin,
            "repair_date": repair_date,
            "repair_type": repair_type,
            "description": description,
            "mileage": mileage,
            "cost": cost,
            "dealer_name": dealer,
            "warranty_claim": warranty_claim
        })

# Create DataFrame
repairs_df = pd.DataFrame(repairs_data)

# Display sample
print(f"✓ Generated {len(repairs_df)} repair records")
print("\nSample data:")
display(repairs_df.head(10))

# Compare Toyota Camry vs other vehicles
camry_vins = vehicles_df[(vehicles_df['make'] == 'Toyota') & (vehicles_df['model'] == 'Camry')]['vin'].tolist()
camry_repairs = repairs_df[repairs_df['vin'].isin(camry_vins)]
other_repairs = repairs_df[~repairs_df['vin'].isin(camry_vins)]

camry_avg = len(camry_repairs) / len(camry_vins) if camry_vins else 0
other_avg = len(other_repairs) / (len(vins) - len(camry_vins)) if (len(vins) - len(camry_vins)) > 0 else 0

print(f"\n📊 Repair Comparison (Toyota Camry vs Others):")
print("-" * 60)
print(f"  Toyota Camry: {len(camry_repairs)} repairs across {len(camry_vins)} vehicles ({camry_avg:.2f} avg repairs/vehicle)")
print(f"  Other Vehicles: {len(other_repairs)} repairs across {len(vins) - len(camry_vins)} vehicles ({other_avg:.2f} avg repairs/vehicle)")
print(f"  Camry has {((camry_avg / other_avg) - 1) * 100:.1f}% more repairs than average" if other_avg > 0 else "")


In [ ]:
# Cell 5b: Insert Repair Records into Snowflake using Snowpark
# ==============================================================

# Rename columns to match table schema (uppercase for Snowflake)
repairs_sf = repairs_df.rename(columns={
    'vin': 'VIN',
    'repair_date': 'REPAIR_DATE',
    'repair_type': 'REPAIR_TYPE',
    'description': 'DESCRIPTION',
    'mileage': 'MILEAGE',
    'cost': 'COST',
    'dealer_name': 'DEALER_NAME',
    'warranty_claim': 'WARRANTY_CLAIM'
})

# Write DataFrame to Snowflake table using Snowpark
session.write_pandas(
    repairs_sf, 
    table_name="REPAIR_RECORDS",
    database=DATABASE_NAME,
    schema=SCHEMA_NAME,
    auto_create_table=False,
    overwrite=True
)

print(f"✓ Inserted {len(repairs_df)} repair records into REPAIR_RECORDS table using Snowpark")

# Final Summary
print("\n" + "=" * 60)
print("✓ Synthetic Data Generation Complete!")
print("=" * 60)
print(f"  Vehicles:       {len(vehicles_df)} records")
print(f"  Owners:         {len(owners_df)} records")
print(f"  Repair Records: {len(repairs_df)} records")
print("=" * 60)

# Verify data in tables
print("\n📊 Verification Queries:")
print(session.sql("SELECT COUNT(*) as VEHICLE_COUNT FROM VEHICLES").collect())
print(session.sql("SELECT COUNT(*) as OWNER_COUNT FROM OWNERS").collect())
print(session.sql("SELECT COUNT(*) as REPAIR_COUNT FROM REPAIR_RECORDS").collect())
